# Simulations with OSN adaptation

Same model of OSN saturation as in ``nonlinear_osn_turbulent_illustration.ipynb``, 

$$ s_i(t) = F_\mathrm{max} \frac{\sum_\gamma K_{i \gamma} c_\gamma}{\exp{(\epsilon_i(t))} + \sum_\gamma K_{i \gamma} c_\gamma} $$

but with modified IBCM and BioPCA integration functions that promote $\epsilon_i(t)$ to dynamical variables with feedback from OSN activity $s_i(t)$ and a target amplitude

$$ \frac{\mathrm{d} \epsilon_i(t)}{\mathrm{d} t} = \frac{1}{\tau_\mathrm{a}} \left( s_i(t) - s_{i, 0} \right) $$

where we also clip (i.e. stop updating beyond this range) $\epsilon \in [\epsilon_L, \epsilon_H]$ to prevent divergences arising from a continued excess or deficit of OSN activity. In other words, adaptation only occurs on a finite range.  


## Functions of general interest

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter
import os, json
from os.path import join as pj
import sys
if "../" not in sys.path:
    sys.path.insert(1, "../")

from modelfcts.ibcm import (
    ibcm_respond_new_odors,  # Unchanged if we give inputs nonlinearized with correct epsilon
    compute_mbars_cgammas_cbargammas,  # Unchanged
)
from modelfcts.biopca import (
    build_lambda_matrix,  
    biopca_respond_new_odors  # Unchanged if we give nonlinearized inputs with correct epsilon
)
# Do not consider average or idealized subtraction here

from modelfcts.checktools import (
    check_conc_samples_powerlaw_exp1,
    analyze_pca_learning  # Unchanged if give pre-computed nonlinear inputs
)
from utils.metrics import jaccard, l2_norm
from modelfcts.distribs import (
    truncexp1_inverse_transform, 
    truncexp1_density, 
    truncexp1_average,
    powerlaw_cutoff_inverse_transform,
    inverse_transform_tanhcdf
)
# re-use functions for nonlinear OSNs, will need to put 
# updated epsilon in back_params at each step
from modelfcts.nonlin_adapt_osn import (  
    generate_odor_tanhcdf, 
    combine_odors_affinities, 
    update_powerlaw_times_concs_affinities,
    sample_background_powerlaw_nl_osn
)
from modelfcts.backgrounds import (  #
    logof10, 
    sample_ss_conc_powerlaw,   # unchanged
    update_tc_odor  # unchanged
)
from modelfcts.tagging import (  # unchanged
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray, 
)
from utils.statistics import seed_from_gen
from utils.smoothing_function import moving_average

from simulfcts.plotting import (
    plot_cbars_gammas_sums, 
    plot_cbars_gamma_series, 
    plot_3d_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats

# Initialization

In [ ]:
do_save_plots = False
do_save_outputs = False

root_dir = pj("..")
outputs_folder = pj(root_dir, "results", "for_plots", "nonlin_adapt")
panels_folder = pj(root_dir, "figures", "nonlin_adapt")
params_folder = pj(root_dir, "results", "common_params")

# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)
    
models = list(model_colors.keys())

In [ ]:
models = ["ibcm", "biopca", "avgsub",  "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "orthogonal": "Orthogonal",
    "none": "None",
    "optimal": "Optimal"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "orthogonal": "xkcd:pale rose",
    "none": "grey",
    "optimal": "xkcd:powder blue"
}

# Main new simulation functions

In [ ]:
def integrate_ibcm_adaptation(vari_inits, update_bk, bk_init,
    ibcm_params, inhib_params, bk_params, adapt_params, tmax, dt,
    seed=None, noisetype="normal", skp=1, **options):
    r""" See docs of integrate_inhib_ibcm_network_options. Differences:
    
    Args:
        vari_inits, update_bk, bk_init, ibcm_params, inhib_params, bk_params, 
        adapt_params, tmax, dt, seed=None, noisetype="normal", skp=1, **options
        
        adapt_params (list of 3 floats, 1 vector): epsilon adaptation time scale, 
            lower and upper limits on epsilon, target osn activities.  
            
        Moreover, we assume that bk_params[-2] is the vector of epsilons
    
    Returns:
        [tseries, bk_series, bkvec_series, eps_series, m_series,
        cbar_series, theta_series, w_series, y_series]
        
        eps_series: shaped [n_times, n_s], the valud of each OSN type's
            epsilon at each time point. 
    """
    # Get some of the keyword arguments
    saturation = options.get("saturation", "linear")
    variant = options.get("variant", "intrator")
    activ_fct = str(options.get("activ_fct", "ReLU")).lower()
    decay = options.get("decay", False)
    w_norms = options.get("w_norms", (2, 2))

    # Legacy option to just pass initial M
    if isinstance(vari_inits, np.ndarray):
        m_init = vari_inits
        n_neu = m_init.shape[0]  # Number of neurons
        n_orn = m_init.shape[1]
        w_init = np.zeros([n_orn, n_neu])
        theta_init = None
    elif isinstance(vari_inits, list) and len(vari_inits) == 1:
        m_init = np.asarray(vari_inits[0])
        n_neu = m_init.shape[0]  # Number of neurons
        n_orn = m_init.shape[1]
        w_init = np.zeros([n_orn, n_neu])
        theta_init = None
    else:
        m_init, theta_init, w_init = vari_inits
        n_neu = m_init.shape[0]  # Number of neurons
        n_orn = m_init.shape[1]

    bk_vari_init, bk_vec_init = bk_init
    assert n_orn == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    alpha, beta = inhib_params
    learnrate, tavg, coupling, lambd, sat, ktheta, decay_relative = ibcm_params
    # Compensate for lambda different from 1, if applicable
    mu_abs = learnrate / lambd

    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt*skp)

    # Check that the biggest matrices, W or M, will not use too much memory
    if tseries.shape[0] * n_orn * n_neu > 5e8 / 8:  # 500 MB per series max
        raise ValueError("Excessive memory use by saved series; increase skp")

    # Containers for the solution over time
    bk_series = np.zeros([tseries.shape[0]] + list(bk_vari_init.shape))
    m_series = np.zeros([tseries.shape[0], n_neu, n_orn])
    cbar_series = np.zeros([tseries.shape[0], n_neu])
    w_series = np.zeros([tseries.shape[0], n_orn, n_neu])  # Inhibitory weights
    bkvec_series = np.zeros([tseries.shape[0], n_orn])  # Input vecs, convenient to compute inhibited output
    y_series = np.zeros([tseries.shape[0], n_orn])
    theta_series = np.zeros([tseries.shape[0], n_neu])

    ## Initialize running variables, separate from the containers above to avoid side effects.
    m = m_init.copy()
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    c = m.dot(bkvec)  # un-inhibited neuron activities
    # Initialize neuron activity with m and background at time zero
    cbar = c - coupling*(np.sum(c) - c)  # -c to cancel the subtraction of c[i] itself
    if saturation == "tanh":
        sat_abs = sat * lambd
        cbar = sat_abs * np.tanh(cbar / sat_abs)
    else: sat_abs = None
    if theta_init is None:
        # Important to initialize cbar2_avg to non-zero values, because we divide by this!
        cbar2_avg = np.maximum(cbar*cbar / lambd, learnrate*lambd)
    else:
        cbar2_avg = theta_init.copy()
    wmat = w_init.copy()
    yvec = bk_vec_init - wmat.dot(cbar)
    if activ_fct == "relu":
        relu_inplace(yvec)
    elif activ_fct == "identity":
        pass
    else:
        raise ValueError("Unknown activation fct: {}".format(activ_fct))
        
    # New parameters and initialization for nonlinear OSN model, epsilon
    tau_eps, eps_min, eps_max, osn_targets = adapt_params
    eps_series = np.zeros([tseries.shape[0], n_orn])
    # epsilon gets initialized to midpoint between min and max
    epsvec = np.full(n_orn, 0.5*(eps_min + eps_max))
    assert bk_params[-2].shape == epsvec.shape, "Ensure vector of epsilons is in bk_params[-2]"
    bk_params[-2] = epsvec

    # Store back some initial values in containers
    cbar_series[0] = cbar
    bk_series[0] = bk_vari
    m_series[0] = m_init
    bkvec_series[0] = bkvec
    y_series[0] = yvec
    theta_series[0] = cbar2_avg
    w_series[0] = wmat
    eps_series[0] = epsvec

    # Generate noise samples in advance, by chunks of at most 2e7 samples
    if noisetype == "normal":
        sample_fct = rng.standard_normal
    elif noisetype == "uniform":
        sample_fct = rng.random
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))
    max_chunk_size = int(2e7)
    # step multiple at which we run out of noises and need to replenish
    kchunk = max_chunk_size // bk_vari.size
    max_n_steps = len(tseries)*skp-1  # vs total number of steps

    t = 0
    newax = np.newaxis
    for k in range(0, max_n_steps):
        t += dt
        # Replenish noise samples if necessary
        if k % kchunk == 0:
            steps_left = max_n_steps - k
            noises = sample_fct(size=(min(kchunk, steps_left), *bk_vari.shape))
        
        ### Inhibitory  weights
        # They depend on cbar and yvec at time step k, which are still in cbar, yvec
        # cbar, shape [n_neu], should broadcast against columns of wmat,
        # while yvec, shape [n_orn], should broadcast across rows (copied on each column)
        if w_norms[0] == 2:  # default L2 norm, nice and smooth
            alpha_term = alpha * cbar[newax, :] * yvec[:, newax]
        elif w_norms[0] == 1:  # L1 norm
            aynorm = alpha * l1_norm(yvec)
            alpha_term = aynorm * cbar[newax, :] * np.sign(yvec[:, newax])
        elif w_norms[0] > 2:  # Assuming some Lp norm with p > 2
            # Avoid division by zero for p > 2 by clipping ynorm
            ynorm = max(1e-9, lp_norm(yvec, p=w_norms[0]))
            yterm = np.sign(yvec) * np.abs(yvec/ynorm)**(w_norms[0]-1) * ynorm
            alpha_term = alpha * cbar[newax, :] * yterm[:, newax]
        else:
            raise ValueError("Cannot deal with Lp norms with p < 0 or non-int")

        if w_norms[1] == 2:
            beta_term = beta * wmat
        elif w_norms[1] == 1:
            beta_term = beta * l1_norm(wmat.ravel()) * np.sign(wmat)
        elif w_norms[1] > 2:
            wnorm = max(1e-9, lp_norm(wmat.ravel(), p=w_norms[1]))
            wterm = np.sign(wmat) * np.abs(wmat/wnorm)**(w_norms[1]-1)
            beta_term = beta * wterm * wnorm
        else:
            raise ValueError("Cannot deal with Lp norms with p < 0 or non-int")

        wmat += dt * (alpha_term - beta_term)

        ### IBCM neurons
        # Phi function for each neuron.
        if variant == "intrator":
            phiterms_vec = cbar * (cbar - cbar2_avg)
        #  Law and Cooper modification for faster convergence.
        elif variant == "law":
            phiterms_vec = cbar * (cbar - cbar2_avg) / (ktheta + cbar2_avg/lambd)
        else:
            raise ValueError("Unknown variant: {}".format(variant))

        if saturation == "tanh":
            phiterms_vec *=  1.0 - (cbar/sat_abs)**2
        # Now, careful with broadcast: for each neuron (dimension 0 of m and cbar), we need a scalar element
        # of phiterms_vec times the whole bkvec, for dimension 1 of m.
        # This can be done vectorially with a dot product (n_neu, 1)x(1, n_components)
        rhs_scalar = phiterms_vec - coupling*(np.sum(phiterms_vec) - phiterms_vec)
        # Euler integrator and learning rate
        # learnrate_t = learnrate if t < 150000 else learnrate / 5
        # Reducing learning rate after a while may help.
        # Consider feedback on mu through some metric of how well neurons
        # are inhibiting the background, e.g. s average activity.
        m += mu_abs*dt*rhs_scalar[:, np.newaxis].dot(bkvec[np.newaxis, :])
        # In principle, should add low decay to background subspace
        # To make sure 1) only learn the background space, 2) de-habituate after
        # The decay term is proportional to m, not m^2 like the IBCM term
        # so we needed to divide learnrate by Lambda for the IBCM term
        # but not for this linear decay term, which should use just learnrate
        if decay and variant == "law":
            m -= dt * decay_relative * learnrate / (ktheta + cbar2_avg[:, np.newaxis]/lambd) * m
        elif decay and variant == "intrator":
            m -= dt * decay_relative * learnrate * m
        # Now, update to time k+1 the threshold (cbar2_avg) using cbar at time k
        # to be used to update m in the next time step
        cbar2_avg += dt * (cbar*cbar / lambd - cbar2_avg)/tavg
        
        # Adapt OSNs in response to background at time t
        epsvec += dt / tau_eps * (bkvec - osn_targets)
        epsvec = np.clip(epsvec, a_min=eps_min, a_max=eps_max)

        # Update background to time k+1, to be used in next time step
        bkvec, bk_vari = update_bk(bk_vari, bk_params, noises[k % kchunk], dt)
        
        # Store updated epsilon in bk_params for the next background update
        bk_params[-2] = epsvec

        # Then, compute activity of IBCM neurons at next time step, k+1,
        # with the updated background and synaptic weight vector m
        # Compute un-inhibited activity of each neuron with current input (at time k)
        # With many simulations in parallel, there seemed to be a bottleneck here
        # and also at yvec calculation: turns out it's because of BLAS multithreading
        # So for multiprocessing, this function should be launched in a threadpool_limits
        c = m.dot(bkvec)
        cbar = c - coupling*(np.sum(c) - c)  # -c to cancel the subtraction of c[i] itself
        if saturation == "tanh":
            cbar = sat_abs * np.tanh(cbar / sat_abs)
        # np.sum(c) is a scalar and c a vector, so it broadcasts properly.

        # Lastly, projection neurons at time step k+1
        yvec = bkvec - wmat.dot(cbar)
        if activ_fct == "relu":
            relu_inplace(yvec)

        # Save current state only if at a multiple of skp
        if (k % skp) == (skp - 1):
            knext = (k+1) // skp
            w_series[knext] = wmat
            m_series[knext] = m
            bk_series[knext] = bk_vari
            bkvec_series[knext] = bkvec
            cbar_series[knext] = cbar  # Save activity of neurons at time k+1
            y_series[knext] = yvec
            theta_series[knext] = cbar2_avg
            eps_series[knext] = epsvec

    return [tseries, bk_series, bkvec_series, eps_series, m_series,
            cbar_series, theta_series, w_series, y_series]


## BioPCA

In [ ]:
def integrate_biopca_adaptation(ml_inits, update_bk, bk_init,
                biopca_params, inhib_params, bk_params, adapt_params, tmax, dt,
                seed=None, noisetype="normal", skp=1, **model_options):
    """
     See docs of integrate_inhib_biopca_network_skip. Differences:
    
    Args:
        ml_inits, update_bk, bk_init, biopca_params, inhib_params, bk_params, 
        adapt_params, tmax, dt, seed=None, noisetype="normal", skp=1, **model_options
        
        adapt_params (list of 3 floats, 1 vector): epsilon adaptation time scale, 
            lower and upper limits on epsilon, target osn activities.  
            
        Moreover, we assume that bk_params[-2] is the vector of epsilons
    
    Returns:
        [tseries, bk_series, bkvec_series, eps_series, m_series,
        cbar_series, theta_series, w_series, y_series]
        
        eps_series: shaped [n_times, n_s], the valud of each OSN type's
            epsilon at each time point. 
    """
    remove_mean = model_options.get("remove_mean", False)
    remove_lambda = model_options.get("remove_lambda", False)
    activ_fct = str(model_options.get("activ_fct", "ReLU")).lower()
    w_norms = model_options.get("w_norms", (2, 2))
    m_init, l_init = ml_inits
    # Note: keep lambda matrix as 1d diagonal only, replace dot products by:
    # Lambda.dot(A): Lambda_ii applied to row i, replace by Lambda_diag[:, None]*A element-wise
    # A.dot(Lambda): Lambda_ii applied to column i, just A*Lambda broadcasts right
    n_neu = m_init.shape[0]  # Number of neurons N_I
    n_orn = m_init.shape[1]  # Number of input neurons N_ORN
    bk_vari_init, bk_vec_init = bk_init
    assert n_orn == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    alpha, beta = inhib_params
    # xrate will be a dummy value if remove_mean == False
    mrate, lrate, lambda_max, lambda_range, xrate = biopca_params
    lrate_l = lrate / lambda_max**2

    # Choose Lambda diagonal matrix as advised in Minden et al., 2018
    lambda_diag = build_lambda_matrix(lambda_max, lambda_range, n_neu)
    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt*skp)

    # Check that the biggest matrices, W or M, will not use too much memory
    if tseries.shape[0] * n_orn * n_neu > 5e8 / 8:  # 500 MB per series max
        raise ValueError("Excessive memory use by saved series; increase skp")

    # Containers for the solution over time
    bk_series = np.zeros([tseries.shape[0]] + list(bk_vari_init.shape))
    m_series = np.zeros([tseries.shape[0], n_neu, n_orn])  # series of M^T (N_IxN_D)
    l_series = np.zeros([tseries.shape[0], n_neu, n_neu])  # series of L (N_IxN_I)
    cbar_series = np.zeros([tseries.shape[0], n_neu])  # series of projections
    w_series = np.zeros([tseries.shape[0], n_orn, n_neu])  # Inhibitory weights W (N_DxN_I)
    bkvec_series = np.zeros([tseries.shape[0], n_orn])  # Input vecs, convenient to compute inhibited output
    y_series = np.zeros([tseries.shape[0], n_orn])  # series of proj. neurons
    if remove_mean:
        xmean_series = np.zeros([tseries.shape[0], n_orn])
    else:
        xmean_series = None

    ## Initialize running variables, separate from the containers above to avoid side effects.
    c = np.zeros(n_neu)  # un-inhibited neuron activities (before applying L)
    cbar = np.zeros(n_neu)  # inhibited neuron activities (after applying L)
    wmat = w_series[0].copy()  # Initialize with null inhibition
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    mmat = m_init.copy()
    lmat = l_init.copy()
    yvec = bk_vec_init.copy()
    if activ_fct == "relu":
        relu_inplace(yvec)
    elif activ_fct == "identity":
        pass
    else:
        raise ValueError("Unknown activation function: {}".format(activ_fct))
    if remove_mean:
        xmean = np.zeros(bkvec.shape)
    else:
        xmean = 0.0

    # Inverse of diagonal of L is used a few times per iteration
    # Indices to access diagonal and off-diagonal elements of L
    # Will be used often, so prepare in advance. Replace dot product
    # with diagonal matrix by element-wise products.
    diag_idx = np.diag_indices(l_init.shape[0])
    inv_l_diag = 1.0 / l_init[diag_idx]  # 1d flattened diagonal
    # Use this difference the only time M_d is needed per iteration
    # Faster to re-invert inv_l_diag than to slice lmat again
    # l_offd = lmat - dflt(1.0 / inv_l_diag)  # is faster than
    # l_offd = lmat - dflt(lmat[diag_idx])
    newax = np.newaxis
    dflt = np.diagflat

    # Initialize neuron activity with m and background at time zero
    c = inv_l_diag * (mmat.dot(bkvec - xmean))
    cbar = c - inv_l_diag*np.dot(lmat-dflt(1.0 / inv_l_diag), c)
    if remove_lambda:
        cbar = cbar / lambda_diag
        
    # New parameters and initialization for nonlinear OSN model, epsilon
    tau_eps, eps_min, eps_max, osn_targets = adapt_params
    eps_series = np.zeros([tseries.shape[0], n_orn])
    # epsilon gets initialized to midpoint between min and max
    epsvec = np.full(n_orn, 0.5*(eps_min + eps_max))
    assert bk_params[-2].shape == epsvec.shape, "Ensure vector of epsilons is in bk_params[-2]"
    bk_params[-2] = epsvec

    # Store back some initial values in containers
    cbar_series[0] = cbar
    bk_series[0] = bk_vari
    m_series[0] = m_init
    l_series[0] = l_init
    bkvec_series[0] = bkvec
    y_series[0] = yvec
    eps_series[0] = epsvec
    
    # Generate noise samples in advance, by chunks of at most 2e7 samples
    if noisetype == "normal":
        sample_fct = rng.standard_normal
    elif noisetype == "uniform":
        sample_fct = rng.random
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))
    max_chunk_size = int(2e7)
    # step multiple at which we run out of noises and need to replenish
    kchunk = max_chunk_size // bk_vari.size
    max_n_steps = len(tseries)*skp-1  # vs total number of steps

    t = 0
    newax = np.newaxis
    for k in range(0, max_n_steps):
        # Replenish noise samples if necessary
        if k % kchunk == 0:
            steps_left = max_n_steps - k
            noises = sample_fct(size=(min(kchunk, steps_left), *bk_vari.shape))
        
        # Learning the mean: independent of everything else.
        if remove_mean:
            xmean = xmean + dt * xrate * (bkvec - xmean)
        # Else, xmean stays 0

        ### Inhibitory  weights
        # They depend on cbar and yvec at time step k, which are still in cbar, yvec
        # cbar, shape [n_neu], should broadcast against columns of wmat,
        # while yvec, shape [n_orn], should broadcast across rows (copied on each column)
        if w_norms[0] == 2:  # default L2 norm, nice and smooth
            alpha_term = alpha * cbar[newax, :] * yvec[:, newax]
        elif w_norms[0] == 1:  # L1 norm
            aynorm = alpha * l1_norm(yvec)
            alpha_term = aynorm * cbar[newax, :] * np.sign(yvec[:, newax])
        elif w_norms[0] > 2:  # Assuming some Lp norm with p > 2
            # Avoid division by zero for p > 2 by clipping ynorm
            ynorm = max(1e-9, lp_norm(yvec, p=w_norms[0]))
            yterm = np.sign(yvec) * np.abs(yvec/ynorm)**(w_norms[0]-1) * ynorm
            alpha_term = alpha * cbar[newax, :] * yterm[:, newax]
        else:
            raise ValueError("Cannot deal with Lp norms with p < 0 or non-int")

        if w_norms[1] == 2:
            beta_term = beta * wmat
        elif w_norms[1] == 1:
            beta_term = beta * l1_norm(wmat.ravel()) * np.sign(wmat)
        elif w_norms[1] > 2:
            wnorm = max(1e-9, lp_norm(wmat.ravel(), p=w_norms[1]))
            wterm = np.sign(wmat) * np.abs(wmat/wnorm)**(w_norms[1]-1)
            beta_term = beta * wterm * wnorm
        else:
            raise ValueError("Cannot deal with Lp norms with p < 0 or non-int")

        wmat += dt * (alpha_term - beta_term)

        ### Online PCA weights
        # Synaptic plasticity: update mmat, lmat to k+1 based on cbar at k
        mmat += dt * mrate * (cbar[:, newax].dot(bkvec[newax, :]) - mmat)
        lmat += dt * mrate * lrate_l * (cbar[:, newax].dot(cbar[newax, :])
                        - lambda_diag[:, newax] * lmat * lambda_diag)
        # Update too the variable saving the inverse of the diagonal of L
        inv_l_diag = 1.0 / lmat[diag_idx]

        t += dt
        
        # Adapt OSNs in response to background at time t
        epsvec += dt / tau_eps * (bkvec - osn_targets)
        epsvec = np.clip(epsvec, a_min=eps_min, a_max=eps_max)

        # Update background to time k+1, to be used in next time step (k+1)
        bkvec, bk_vari = update_bk(bk_vari, bk_params, noises[k % kchunk], dt)
        
        # Store updated epsilon in bk_params for the next background update
        bk_params[-2] = epsvec

        # Neural dynamics (two-step) at time k+1, to be used in next step
        c = inv_l_diag * (mmat.dot(bkvec - xmean))  # L_d^(-1) M^T x
        # Lateral inhibition between neurons
        cbar = c - inv_l_diag*np.dot(lmat - dflt(1.0/inv_l_diag), c)
        if remove_lambda:
            # Remove the Lambda scale of eigenvectors, so the W matrix does
            # not need to compensate too much.
            # So we use Lambda^{-1}L^{-1}M as a projector as prescribed in Minden 2018
            cbar = cbar / lambda_diag

        # Lastly, projection neurons at time step k+1.
        # xmean is 0 if we don't remove the mean
        yvec = bkvec - xmean - wmat.dot(cbar)
        if activ_fct == "relu":
            relu_inplace(yvec)

        # Save current state only if at a multiple of skp
        if (k % skp) == (skp - 1):
            knext = (k+1) // skp
            w_series[knext] = wmat
            m_series[knext] = mmat
            l_series[knext] = lmat
            bk_series[knext] = bk_vari
            bkvec_series[knext] = bkvec
            cbar_series[knext] = cbar  # Save activity of neurons at time k+1
            y_series[knext] = yvec
            eps_series[knext] = epsvec
            if remove_mean:
                xmean_series[knext] = xmean
    return (tseries, bk_series, bkvec_series, eps_series, m_series, l_series,
                xmean_series, cbar_series, w_series, y_series)


# Parameters

## Common parameters

In [ ]:
# Initialize common simulation parameters
n_dimensions = 50  # Fly number
n_components = 6  # Number of background odors

# Common parameters for toy and full simulations
inhib_rates = [0.00005, 0.00001]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration
duration = 360000.0
deltat = 1.0

# Simulation skipping, 50 is enough for plots
skp = 50 * int(1.0 / deltat)
tser_common = np.arange(0.0, duration, deltat*skp)

# Common model options
activ_function = "identity"  #"ReLU"

# Background process
combine_fct = combine_odors_affinities
update_fct = update_powerlaw_times_concs_affinities

# Scale of affinity vectors: default
kscale = 5e-4  # default is 5e-4

# OSN target activity and epsilon ranges
# TODO; maybe adjust given the odor vectors, f_max scale, etc. 
target_osn_activ = np.full(n_dimensions, 1.0 / np.sqrt(n_dimensions))
adaptation_params = [
    25.0,  # tau_adapt = 250 ms
    1.0,  # eps_min, allow quite low
    10.0,  # eps_max
    target_osn_activ 
]

## Background initialization functions

In [ ]:
def default_background_params(n_comp):
    """ Default time and concentration parameters for the turbulent process"""
    # Turbulent background parameters: same rates and constants for all odors
    back_pms_turbulent = [
        np.asarray([1.0] * n_comp),        # whiff_tmins
        np.asarray([500.] * n_comp),       # whiff_tmaxs
        np.asarray([1.0] * n_comp),        # blank_tmins
        np.asarray([800.0] * n_comp),      # blank_tmaxs
        np.asarray([0.6] * n_comp),        # c0s
        np.asarray([0.5] * n_comp),        # alphas
    ]
    return back_pms_turbulent

In [ ]:
def initialize_back_params(adapt_params, rgen, n_comp, n_dim):
    # Turbulent background parameters: same rates and constants for all odors
    back_pms = default_background_params(n_comp)
    
    tau_eps, eps_min, eps_max, osn_targets = adapt_params
    epsils_vec = np.full(n_dim, 0.5 * (eps_min + eps_max))
    back_comps = generate_odor_tanhcdf((n_comp, n_dim), rgen, unit_scale=kscale)

    # To keep OSN amplitudes comparable to usual simulations, scale down OSN max. ampli
    avg_whiff_conc = np.mean(truncexp1_average(*back_pms[4:6]))
    
    # Same adjustment of the OSN amplitude as in the performance recognition tests
    raw_conc_factor = 2.5
    raw_ampli = 2.5
    np_statistic = np.mean  # np.mean, np.median, np.amax

    raw_osn_activ = np_statistic(combine_fct(np.full(n_comp, raw_conc_factor * avg_whiff_conc), 
                                        back_comps, epsils_vec, fmax=1.0))
    max_osn_ampli = raw_ampli / (raw_osn_activ * np.sqrt(n_dim))

    # Add these extra parameters to the list of background params
    back_pms.append(max_osn_ampli)
    back_pms.append(epsils_vec)
    back_pms.append(back_comps)

    # Initialization
    # Initial values of background process variables (t, c for each variable)
    init_concs = sample_ss_conc_powerlaw(*back_pms[:-3], size=1, rgen=rgen)
    init_times = powerlaw_cutoff_inverse_transform(
                    rgen.random(size=n_comp), *back_pms[2:4])
    tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)

    # Initial background vector: combine odors with the tc_init concentrations
    init_bkvec = combine_fct(tc_init[:, 1], back_comps, epsils_vec, fmax=max_osn_ampli)
    # nus are first in the list of initial background params
    init_back = [tc_init, init_bkvec]
    
    return back_pms, init_back

## IBCM habituation and simulation parameters

In [ ]:
# IBCM model parameters, same for each tested epsilon
n_i_ibcm = 24  # Number of inhibitory neurons for IBCM case

# Model rates
learnrate_ibcm = 0.001  #5e-5
tau_avg_ibcm = 1600  # 2000
coupling_eta_ibcm = 0.7/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.5
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "law", 
    "decay": True
}

### Function to run and clean a simulation

Uses global IBCM parameters defined above. 

In [ ]:
def run_ibcm_simulation_adapt(adapt_params, n_comp, n_dim, rgenseed, simseed, skp_local=skp):
    print("Initializing IBCM simulation for adapt_params[:3] =", adapt_params[:3])
    # Initialize background with the random generator with seed rgenseed
    rgen = np.random.default_rng(rgenseed)
    res = initialize_back_params(adapt_params, rgen, n_comp, n_dim)
    back_params_local, init_back = res
    
    # Initial synaptic weights: small positive noise
    init_synapses_ibcm = 0.2*rgen.standard_normal(size=[n_i_ibcm, n_dim])*lambd_ibcm
    
    # Run the IBCM simulation
    print("Starting IBCM simulation...")
    tstart = perf_counter()
    sim_results = integrate_ibcm_adaptation(
                init_synapses_ibcm, update_fct, init_back, 
                ibcm_rates, inhib_rates, back_params_local, 
                adapt_params, duration, deltat, seed=simseed, 
                noisetype="uniform",  skp=skp_local, **ibcm_options
    )
    tend = perf_counter()
    print("Finished IBCM simulation in {:.2f} s".format(tend - tstart))
    
    return back_params_local, sim_results

In [ ]:
def mix_new_odors_in_manifold(back_pms, conc_ser, new_conc_rel, rgen, n_ex=2, n_samp=10):
    """Mix n_ex new odors with n_samp background samples each. 
    Returns a 3d-array of mixtures, indexed [n_ex, n_samp, n_dim], 
    and the new odor vectors, a 2d array indexed [n_ex, n_dim]. 
    """
    back_odors = back_pms[-1]
    n_comp, n_dim = back_odors.shape[0], back_odors.shape[1]
    max_ampli = back_pms[-3]
    new_odors = generate_odor_tanhcdf((n_ex, n_dim), rgen, unit_scale=kscale)
    avg_whiff_conc = np.mean(truncexp1_average(*back_pms[4:6]))
    new_conc = avg_whiff_conc * new_conc_rel
    non_null_concs = conc_ser[np.any(conc_ser > 0.0, axis=1)]
    epsils_vec = back_pms[-2]
    back_concs = non_null_concs[rgen.choice(non_null_concs.shape[0], size=n_ex*n_samp, replace=True)]
    back_concs = back_concs.reshape(n_ex, n_samp, n_comp)
    all_mixed_samples = []
    for i in range(n_ex):
        joint_kmats = np.concatenate([back_odors, new_odors[i:i+1]], axis=0)
        mixed_samples_i = []
        for j in range(n_samp):
            joint_concs = np.concatenate([back_concs[i, j:j+1], np.full((1, 1), new_conc)], axis=1)
            mixed_samples_i.append(combine_fct(joint_concs, joint_kmats, epsils_vec, fmax=max_ampli))
        mixed_samples_i = np.concatenate(mixed_samples_i, axis=0)
        all_mixed_samples.append(mixed_samples_i)
        
    return np.stack(all_mixed_samples, axis=0), new_odors

In [ ]:
# Cleaning function
def analyze_clean_ibcm_simul(results_raw, back_pms, rgenseed, n_ex=2, n_samp=10, t_mix=-1):
    """
    Args:
        results_raw = (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
            cbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm)
    Returns:
        cbars_gamma, wser_ibcm, bkvecser_ibcm, 
            yser_ibcm, moments_conc, cgammas_bar_counts, specif_gammas, correl_c_conc
    """
    (tser_ibcm, nuser_ibcm, bkvecser_ibcm, eps_ser, mser_ibcm, 
        cbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm) = results_raw
    # Calculate cgammas_bar and mbars
    transient = int(5/6*duration / deltat) // skp
    back_components = back_pms[-1]
    basis = back_components / l2_norm(back_components, axis=1)[:, None] 
    

    # Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
    mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(
                                mser_ibcm, coupling_eta_ibcm, basis)
    
    # Moments of concentrations
    conc_ser = nuser_ibcm[:, :, 1]
    mean_conc = np.mean(conc_ser)
    sigma2_conc = np.var(conc_ser)
    thirdmom_conc = np.mean((conc_ser - mean_conc)**3)
    moments_conc = [float(mean_conc), float(sigma2_conc), float(thirdmom_conc)]

    # Count how many dot products are at each possible value. Use cbar = 1.0 as a split. 
    cbars_gamma_mean = np.mean(cbars_gamma[transient:], axis=0)
    specif_gammas = np.argmax(np.mean(cbars_gamma[transient:], axis=0), axis=1)
    
    cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
    conc_ser_centered = conc_ser - np.mean(conc_ser[transient:], axis=0)
    correl_c_conc = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)
    
    ysernorm_ibcm = l2_norm(yser_ibcm, axis=1)
    
    # Examples of mixing new odors with the background
    rgen = np.random.default_rng(np.random.SeedSequence(rgenseed).spawn(2)[1])
    back_pms_local = list(back_pms)
    back_pms_local[-2] = eps_ser[t_mix]
    mixres = mix_new_odors_in_manifold(back_pms_local, conc_ser, 1.0, rgen, n_ex=n_ex, n_samp=n_samp)
    mixed_samples, new_odors = mixres
    results_clean = (cbars_gamma, wser_ibcm, bkvecser_ibcm, eps_ser, ysernorm_ibcm, moments_conc, 
                     cbars_gamma_mean, specif_gammas, correl_c_conc, back_components, 
                     conc_ser, mixed_samples, new_odors)
    return results_clean

In [ ]:
# Plotting function
# Plotting functions for IBCM
def plot_ibcm_results(res_ibcm_raw, res_ibcm_clean):
    (cbars_gamma, wser_ibcm, bkvecser_ibcm, eps_ser, ysernorm_ibcm, 
         moments_conc, cbars_gamma_mean, specif_gammas, correl_c_conc, 
         back_comps, conc_ser, _, _) = res_ibcm_clean

    # Plot of cbars gamma series
    fig , ax, _ = plot_cbars_gamma_series(tser_common, cbars_gamma, 
                            skp=2, transient=320000 // skp)
    fig.tight_layout()
    leg = ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))
    plt.show()
    plt.close()


    # Plots of neuron specificities
    fig, ax = plt.subplots()
    img = ax.imshow(correl_c_conc.T)
    ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
    fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
                 r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
                location="top")
    fig.tight_layout()
    plt.show()
    plt.close()

    # Check if each component has at least one neuron
    print("Odor specificities:", specif_gammas)
    split_val = 2.5
    for comp in range(n_components):
        print("Number of neurons specific to component {}: {}".format(
                comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

    # Plot of background inhibition
    fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                    tser_common, res_ibcm_raw[2], res_ibcm_raw[8], skp=2)

    # Compute noise reduction factor, annotate
    transient = 250000 // skp
    norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

    print("Mean activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
    print("Standard deviation of activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
    ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
               xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

    ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
    fig.tight_layout()
    
    # TODO: plot epsilon vector series?
    
    plt.show()
    plt.close()

## BioPCA habituation and simulation parameters

In [ ]:
# BioPCA model parameters, same for all epsilons
n_i_pca = n_components * 2  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 1e-4  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
# but scale it up to counteract W regularization
lambda_range_pca = 0.5
lambda_max_pca = 9.0
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)

xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca, xavg_rate_pca]

### Function to run and clean a simulation

Uses global BioPCA parameters defined above. 

In [ ]:
def run_biopca_simulation_adapt(adapt_params, n_comp, n_dim, rgenseed, simseed, skp_local=skp):
    print("Initializing BioPCA simulation for adapt_params[:3] =", adapt_params[:3])
    # Initialize background parameters, give same rgenseed as IBCM to have same background
    rgen = np.random.default_rng(rgenseed)
    res = initialize_back_params(adapt_params, rgen, n_comp, n_dim)
    back_params_local, init_back = res
        
    init_synapses_pca = rgen.standard_normal(size=[n_i_pca, n_dim]) / np.sqrt(n_i_pca)
    init_mmat_pca = rgen.standard_normal(size=[n_i_pca, n_dim]) / np.sqrt(n_dim)
    init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
    ml_inits_pca = [init_mmat_pca, init_lmat_pca]
    
    # Run the IBCM simulation
    print("Starting BioPCA simulation...")
    tstart = perf_counter()
    sim_results = integrate_biopca_adaptation(
                ml_inits_pca, update_fct, init_back, biopca_rates, 
                inhib_rates, back_params_local, adapt_params, duration, deltat, 
                seed=simseed, noisetype="uniform", skp=skp_local, **pca_options
    )
    tend = perf_counter()
    print("Finished BioPCA simulation in {:.2f} s".format(tend - tstart))
    
    return back_params_local, sim_results

In [ ]:
def analyze_clean_biopca_simul(results_raw):
    """
    We do not need to save odor vectors (back_components), 
    since the IBCM simulation will provide them for both models. 
    
    Args:
        results_raw = (tser_pca, nuser_pca, bkvecser_pca, mser_pca, 
            lser_pca, xser_pca, cbarser_pca, wser_pca, yser_pca)
    Returns:
        bkvecser_pca, ysernorm_pca, wser_pca, true_pca, 
            learnt_pca, off_diag_l_avg_abs, align_error_ser)
    """
    (tser_pca, nuser_pca, bkvecser_pca, eps_ser, mser_pca, lser_pca, xser_pca, 
         cbarser_pca, wser_pca, yser_pca) = results_raw
    
    # Analyze versus true offline PCA of the background samples
    print("Starting analysis of BioPCA vs true PCA")
    tstart = perf_counter()
    res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
    true_pca, learnt_pca, _, off_diag_l_avg_abs, align_error_ser = res
    tend = perf_counter()
    print("Completed analysis in {:.1f} s".format(tend - tstart))
    
    ysernorm_pca = l2_norm(yser_pca, axis=1)
    bkvecsernorm_pca = l2_norm(bkvecser_pca, axis=1)
    
    # Also save info about background vs yser_pca
    results_clean = (bkvecsernorm_pca, eps_ser, ysernorm_pca, wser_pca,
                     true_pca, learnt_pca, off_diag_l_avg_abs, align_error_ser)
    return results_clean

In [ ]:
def plot_biopca_results(res_biopca_raw, res_biopca_clean):
    (bkvecsernorm_pca, epser_pca, ysernorm_pca, wser_pca, true_pca, 
    learnt_pca, off_diag_l_avg_abs, align_error_ser) = res_biopca_clean

    # Plot learnt vs true PCA
    fig, axes = plot_pca_results(tser_common/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
    axes[-1].set_xlabel("Time (x1000 steps)")
    axes[0].get_legend().remove()
    fig.tight_layout()
    fig.set_size_inches(fig.get_size_inches()[0], 2.5*plt.rcParams["figure.figsize"][1])
    plt.show()
    plt.close()

    # Plot level of background inhibition
    fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                    tser_common, res_biopca_raw[2], res_biopca_raw[9], skp=2)

    # Compute noise reduction factor, annotate
    transient = 250000 // skp
    norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

    print("Mean activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
    print("Standard deviation of activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
    ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
               xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

    ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
    fig.tight_layout()
    plt.show()
    plt.close()

# Additional plotting functions, to visualize the manifold

In [ ]:
# Since we will make similar plots, define functions
def plot_manifold(bkser, bkvecs, conc_ser, view_params, 
                  mixed_new_odors=None, new_odor_vec=None, dims=(0, 1, 2)):
    # Plot 2D manifold in a 3D slice,
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    # Too many combinations for 6 odors, maybe just highlight
    # single-odor axes
    where_each = (conc_ser > 0).astype(bool)
    n_odors = where_each.shape[1]
    locations = {}
    # Track places with 0 or 1 odor
    any_single_odor = np.all(where_each == False, axis=1)  # start with places with 0 odor
    for i in range(n_odors):
        mask = np.zeros((1, n_odors), dtype=bool)
        mask[0, i] = True
        locations["Odor {}".format(i)] = np.all(where_each == mask, axis=1)
        any_single_odor += locations["Odor {}".format(i)]  # add places with odor i only
    locations["2+ odors"] = np.logical_not(any_single_odor)  # 2+ odors anywhere else
    single_odor_colors = sns.color_palette("colorblind", n_colors=n_odors)
    all_colors = {"Odor {}".format(i): single_odor_colors[i] for i in range(n_odors)}
    all_colors["2+ odors"] = "grey"
    
    orig = np.zeros([3, 6])
    locations_order = ["2+ odors"] + ["Odor {}".format(i) for i in range(n_odors)]
    for lbl in locations_order:
        alpha = 0.3 if lbl.startswith("2+") else 1.0
        slc = locations[lbl]
        tskp = 5 if lbl.startswith("2+") else 1
        zshift = 0.03 if lbl.startswith("2+") else 0.0
        shift = 0.03 if lbl.startswith("2+") else 0.0
        lbl_append = ""# if lbl.startswith("2+") else " alone"
        bk_subset = [bkser[slc, d].copy() + shift for d in dims]
        bk_subset[2] -= zshift
        ax.scatter(bk_subset[0][::tskp], bk_subset[1][::tskp], bk_subset[2][::tskp], 
                   s=4, lw=0.3, label=lbl+lbl_append, color=all_colors[lbl], alpha=alpha)
    vecs = bkvecs / l2_norm(bkvecs, axis=1)[:, None]
    print(vecs.shape)
    ax.quiver(*orig, *(vecs[:, dims].T), color="k", lw=1.5, arrow_length_ratio=0.2)
    ax.scatter(0, 0, 0, color="k", s=25)
    
    # Also show what adding a new odor can do -- out of the manifold?
    new_odor_lbl = "+ new odor"
    if mixed_new_odors is not None:
        n_new_odors = mixed_new_odors.shape[0]
        new_odors_palette = sns.dark_palette("r", n_colors=n_new_odors+1)[1:]
        for i in range(n_new_odors):
            lbl = new_odor_lbl + " {}".format("abcdefghijklmnop".upper()[i])
            all_colors[lbl] = new_odors_palette[i]
            ax.scatter(mixed_new_odors[i, :, dims[0]], mixed_new_odors[i, :, dims[1]], 
                        mixed_new_odors[i, :, dims[2]], s=6, lw=0.3, 
                        label=lbl+lbl_append, color=all_colors[lbl], alpha=1.0)
            if new_odor_vec is not None:
                vec = new_odor_vec[i] / l2_norm(new_odor_vec[i])
                ax.quiver(*orig, *(vec[list(dims)]), color=all_colors[lbl], 
                          lw=1.5, arrow_length_ratio=0.2)
    # No new odors shown
    else:
        n_new_odors = 0
        

    # Labeling
    for lbl, f in enumerate([ax.set_xlabel, ax.set_ylabel, ax.set_zlabel]):
        # z label gets caught in the zlbl variable at the last iteration
        zlbl = f("OSN {} (of {})".format(lbl+1, bkser.shape[1]), labelpad=-17.5)
    for f in [ax.set_xticks, ax.set_yticks, ax.set_zticks]:
        f([])
    for f in [ax.set_xticklabels, ax.set_yticklabels, ax.set_zticklabels]:
        f([], pad=0.1)
    view_params.setdefault("azim", 240)
    view_params.setdefault("elev", 3)
    ax.view_init(**view_params)
    handles, labels = ax.get_legend_handles_labels()
    # Move the label for 2+ odors to before the new odors
    if n_new_odors > 0:
        handles.insert(-n_new_odors-1, handles[0])
        labels.insert(-n_new_odors-1, labels[0])
    else:
        handles.append(handles[0])
        labels.append(labels[0])
    handles.pop(0)
    labels.pop(0)
    leg = ax.legend(handles=handles, labels=labels, 
        frameon=True, ncol=1, loc="upper left", bbox_to_anchor=(0.85, 1.0), 
        title="Odor presence", title_fontsize=6)
    #loc="upper right", bbox_to_anchor=(0.0, 1.0), frameon=False)
    fig.tight_layout()

    # Need to adjust the tightbox to remove whitespace above and below manually. 
    #ax.set_aspect("equal")
    fig.tight_layout()
    tightbox = fig.get_tightbbox()
    tightbox._bbox.y0 = tightbox._bbox.y0*1.1   #bottom
    tightbox._bbox.y1 = tightbox._bbox.y1 + 0.7*tightbox._bbox.y0  # top
    tightbox._bbox.x0 = tightbox._bbox.x0 * 0.6  # position of left side

    return fig, ax, tightbox

In [ ]:
# Inspect the background process
def pairplots_background(bkser, bkvecs, epsser=None, mixed_new_odors=None, new_odor_vec=None):
    # Background vectors time series with mixed concentrations
    tslice = slice(0, None, 30)
    
    # Scale odor affinities K_{i \gamma} by the average saturation threshold 
    # of the OSN, exp(\epsilon_i), to get the effective affinity scale, 
    # before normalizing each odor vector K_\gamma
    vecs = bkvecs / l2_norm(bkvecs, axis=1)[:, None]
    if epsser is not None:
        mean_eps = np.mean(epsser, axis=0)
        vecs_eff = bkvecs / np.exp(mean_eps[None, :])
        vecs_eff = vecs_eff / l2_norm(vecs_eff, axis=1)[:, None]
    else:
        vecs_eff = None
        
    n_comp = bkvecs.shape[0]
    n_cols = 6
    n_plots = 48 // 2
    n_rows = n_plots // n_cols + min(1, n_plots % n_cols)
    fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True)
    fig.set_size_inches(n_cols*1.0, n_rows*1.0)
    single_odor_colors = sns.color_palette("colorblind", n_colors=n_comp)
    all_colors = {"Odor {}".format(i): single_odor_colors[i] for i in range(n_comp)}
    if mixed_new_odors is not None:
        n_new_odors = mixed_new_odors.shape[0]
        new_odors_palette = sns.dark_palette("r", n_colors=n_new_odors+1)[1:]
    for i in range(n_plots):
        ax = axes.flat[i]
        ax.scatter(bkser[tslice, 2*i+1], bkser[tslice, 2*i], 
                   s=9, alpha=0.5, color="k")
        for j in range(n_comp):
            ax.plot(*zip([0.0, 0.0], vecs[j, 2*i:2*i+2][::-1]), lw=1.5, 
                    color=single_odor_colors[j])
            if epsser is not None:
                ax.plot(*zip([0.0, 0.0], vecs_eff[j, 2*i:2*i+2][::-1]), lw=1.5, ls="--",
                    color=single_odor_colors[j])
        if mixed_new_odors is not None:
            for j in range(n_new_odors):
                clr = new_odors_palette[j]
                ax.scatter(mixed_new_odors[j, :, 2*i+1], mixed_new_odors[j, :, 2*i], 
                       s=6, alpha=1.0, color=clr)
        if new_odor_vec is not None:
            for j in range(n_new_odors):
                vec = new_odor_vec[j] / l2_norm(new_odor_vec[j])
                ax.plot(*zip([0.0, 0.0], vec[2*i:2*i+2][::-1]), lw=2.0, 
                    color=new_odors_palette[j])
        ax.set(xlabel="OSN {}".format(2*i+2), ylabel="OSN {}".format(2*i+1))
    for i in range(n_plots, n_rows*n_cols):
        axes.flat[i].set_axis_off()
    fig.tight_layout()
    
    return fig, axes

In [ ]:
main_seed = 0x7170b82d905839ddda1def555e3a43508
simul_seed = 0x52e7bfc4e1f58395730de6afff855abc

# IBCM
back_ibcm, res_ibcm = run_ibcm_simulation_adapt(adaptation_params, n_components,  
                         n_dimensions, main_seed, simul_seed)
res_ibcm_clean = analyze_clean_ibcm_simul(res_ibcm, back_ibcm, main_seed)

# BioPCA
back_biopca, res_biopca = run_biopca_simulation_adapt(adaptation_params, n_components,
                            n_dimensions, main_seed, simul_seed)
res_biopca_clean = analyze_clean_biopca_simul(res_biopca)

In [ ]:
plot_ibcm_results(res_ibcm, res_ibcm_clean)

In [ ]:
plot_biopca_results(res_biopca, res_biopca_clean)

In [ ]:
# Axes zoom effect from Matplotlib documentation
from matplotlib.transforms import (Bbox, TransformedBbox,
                                   blended_transform_factory)
from mpl_toolkits.axes_grid1.inset_locator import (BboxConnector,
                                                   BboxConnectorPatch,
                                                   BboxPatch)
def connect_bbox(bbox1, bbox2,
                 loc1a, loc2a, loc1b, loc2b,
                 prop_lines, prop_patches=None):
    if prop_patches is None:
        prop_patches = {
            **prop_lines,
            "alpha": prop_lines.get("alpha", 1) * 0.2,
            "clip_on": False,
        }

    c1 = BboxConnector(
        bbox1, bbox2, loc1=loc1a, loc2=loc2a, clip_on=False, **prop_lines)
    c2 = BboxConnector(
        bbox1, bbox2, loc1=loc1b, loc2=loc2b, clip_on=False, **prop_lines)

    bbox_patch1 = BboxPatch(bbox1, **prop_patches, color="grey")
    bbox_patch2 = BboxPatch(bbox2, **prop_patches, color="grey")

    p = BboxConnectorPatch(bbox1, bbox2,
                           loc1a=loc1a, loc2a=loc2a, loc1b=loc1b, loc2b=loc2b,
                           clip_on=False,
                           **prop_patches)

    return c1, c2, bbox_patch1, bbox_patch2, p

def zoom_effect01(ax1, ax2, xmin, xmax, **kwargs):
    """
    Connect *ax1* and *ax2*. The *xmin*-to-*xmax* range in both Axes will
    be marked.

    Parameters
    ----------
    ax1
        The main Axes.
    ax2
        The zoomed Axes.
    xmin, xmax
        The limits of the colored area in both plot Axes.
    **kwargs
        Arguments passed to the patch constructor.
    """

    bbox = Bbox.from_extents(xmin, 0, xmax, 1)

    mybbox1 = TransformedBbox(bbox, ax1.get_xaxis_transform())
    mybbox2 = TransformedBbox(bbox, ax2.get_xaxis_transform())

    prop_patches = {**kwargs, "ec": "none", "alpha": 0.2}

    c1, c2, bbox_patch1, bbox_patch2, p = connect_bbox(
        mybbox1, mybbox2,
        loc1a=3, loc2a=2, loc1b=4, loc2b=1,
        prop_lines=kwargs, prop_patches=prop_patches)

    #ax1.add_patch(bbox_patch1)
    ax2.add_patch(bbox_patch2)
    ax2.add_patch(c1)
    ax2.add_patch(c2)
    ax2.add_patch(p)

    return c1, c2, bbox_patch1, bbox_patch2, p

In [ ]:
# Check what individual OSN epsilons are doing on fast and long time scales
def plot_eps_series(eps_ser, n_shown, tzoom_interv, skp_n=1, smoothsize=201):
    fig = plt.figure()
    gs = fig.add_gridspec(2, 4)
    axes = [fig.add_subplot(gs[0, :3])]
    axes.append(fig.add_subplot(gs[1, :3], sharey=axes[0]))
    axleg = fig.add_subplot(gs[:, 3])
    
    fig.set_size_inches(plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1])
    eps_ser_ibcm = res_ibcm[3]
    eps_ser_ibcm_smooth = moving_average(eps_ser_ibcm, kernelsize=smoothsize, boundary="free")
    tscale = deltat * 10.0 / 1000.0 / 60.0  # minutes
    #eps_ser_biopca = res_biopca[3]

    osn_colors = sns.cubehelix_palette(n_colors=n_shown, 
            start=0.0, rot=1.0, gamma=1.0, hue=0.8, light=0.85, dark=0.15, reverse=True)

    tsl_local = slice(*tzoom_interv, 1)  # limit * skp * 10 = milliseconds, skp=50 default
    tsl_global = slice(0, None, 20)
    n_labels = 6
    skp_lbl = (n_shown // skp_n) // n_labels
    for i in range(0, n_shown, skp_n):
        lbl = "{}".format(i) if (i // skp_n) % skp_lbl == 0 else ""
        axes[0].plot(tser_common[tsl_local]*tscale, eps_ser_ibcm[tsl_local, i], 
                     alpha=0.7, lw=0.75, color=osn_colors[i], label=lbl)
        axes[1].plot(tser_common[tsl_global]*tscale, eps_ser_ibcm_smooth[tsl_global, i], 
                     alpha=0.7, lw=0.75, color=osn_colors[i])

    axes[0].set(ylabel=r"$\epsilon_i(t)$")
    axes[1].set(xlabel="Time (min)", ylabel=r"Smoothed $\epsilon_i(t)$")
    axleg.legend(*axes[0].get_legend_handles_labels(), frameon=False, title="OSN")
    axleg.set_axis_off()
    t1, t2 = tser_common[tzoom_interv[0]]*tscale, tser_common[tzoom_interv[1]]*tscale
    zoom_effect01(axes[0], axes[1], t1, t2, lw=0.8)
    fig.tight_layout(h_pad=-0.1)
    return fig, axes, axleg

In [ ]:
fig, axes, axleg = plot_eps_series(eps_ser_ibcm, 50, (600, 840), skp_n=4, smoothsize=101)
plt.show()
plt.close()

In [ ]:
# (cbars_gamma, wser_ibcm, bkvecser_ibcm, eps_ser, ysernorm_ibcm, moments_conc, 
#                     cbars_gamma_mean, specif_gammas, correl_c_conc, back_components, 
#                     conc_ser, mixed_samples, new_odors)
bkser_ibcm = res_ibcm_clean[2]
bkvecs_ibcm = res_ibcm_clean[9]
concser_ibcm = res_ibcm_clean[10]
fig, ax, box = plot_manifold(bkser_ibcm, bkvecs_ibcm, concser_ibcm, {"azim":220, "elev":30}, dims=(0, 1, 2))
plt.show()
plt.close()

In [ ]:
fig, axes = pairplots_background(bkser_ibcm, bkvecs_ibcm, eps_ser_ibcm)
plt.show()
plt.close()